# Command Class in LangGraph
---

# **1. What is the `Command` Class in LangGraph?**

In **LangGraph**, the **`Command`** class is a **special return type** used by nodes to control **how the graph executes**.

Normally, when a node runs, it **returns data** that updates the state and moves to the **next node automatically**.
But sometimes, you need **more control**:

* ⏸️ **Pause execution** → Wait for human feedback (`Interrupt`)
* 🔀 **Jump to a specific node** → Skip or reroute dynamically
* 🔁 **Loop back** → Retry from the same or previous node
* ⏩ **Skip ahead** → Go to another part of the workflow

This is where the **`Command`** class comes in.
Instead of returning just state updates, a node can **return a `Command` object** that **directs the graph** what to do next.

---

# **2. Command Class Structure**

```python
from langgraph.types import Command, Interrupt

Command(
    update: dict = None,        # State updates (optional)
    goto: str = None,           # Node name to jump to (optional)
    interrupt: Interrupt = None # Interrupt execution and wait for input
)
```

| **Field**   | **Type**    | **Purpose**                                         |
| ----------- | ----------- | --------------------------------------------------- |
| `update`    | `dict`      | Updates state variables before continuing           |
| `goto`      | `str`       | Jumps to a specific node instead of following edges |
| `interrupt` | `Interrupt` | Pauses execution and waits for `.resume()`          |

---

# **3. Example 1 — Simple Use of Command**

Let's make a graph where the agent:

1. Asks a human for approval.
2. If approved, jumps to `"process"`.
3. If rejected, jumps to `"abort"`.

---

### **Code**

```python
from langgraph.graph import StateGraph, END
from langgraph.types import Command, Interrupt
from typing import TypedDict

# Define state structure
class AgentState(TypedDict):
    decision: str
    result: str

# Node where we interrupt for human input
def ask_approval(state: AgentState):
    return Command(
        interrupt=Interrupt(
            message="Do you approve this action? (yes/no)"
        )
    )

# Node when human approves
def process_action(state: AgentState):
    return {"result": "Action approved and processed ✅"}

# Node when human rejects
def abort_action(state: AgentState):
    return {"result": "Action aborted ❌"}

# Decision node based on human input
def decide(state: AgentState):
    if state["decision"].lower() == "yes":
        return Command(goto="process")
    else:
        return Command(goto="abort")

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("ask_approval", ask_approval)
workflow.add_node("decide", decide)
workflow.add_node("process", process_action)
workflow.add_node("abort", abort_action)

workflow.set_entry_point("ask_approval")
workflow.add_edge("ask_approval", "decide")
workflow.add_edge("process", END)
workflow.add_edge("abort", END)

app = workflow.compile()
```

---

### **Running It**

```python
config = {"configurable": {"thread_id": "001"}}

# Step 1 → Invoke graph, pauses at interrupt
result = app.invoke({}, config=config)
print(result)
# Output: Interrupt(message="Do you approve this action? (yes/no)")

# Step 2 → Resume with human feedback
final = app.resume({"decision": "yes"}, config=config)
print(final)
# Output: {'result': 'Action approved and processed ✅'}
```

---

# **4. Example 2 — Command with State Update + Goto**

We can **update state variables** **and** **jump to another node** in one step.

```python
def auto_decide(state: AgentState):
    return Command(
        update={"decision": "yes"},  # Auto-approve
        goto="process"               # Jump straight to processing
    )
```

This is useful when you want the agent to **skip interrupts** under certain conditions.

---

# **5. Example 3 — Command with Interrupt for Human-in-the-Loop**

The most common use of `Command` is with **Interrupts** for **human approval**.

```python
def confirm_booking(state: AgentState):
    return Command(
        interrupt=Interrupt(
            message=f"Do you want to book this flight for ₹{5000}?"
        )
    )
```

When you call `.invoke()`, the graph **pauses** here and waits for `.resume()`:

```python
result = app.invoke({}, config=config)
print(result)
# Output: Interrupt(message="Do you want to book this flight for ₹5000?")

# Later → resume with human approval
final = app.resume({"decision": "yes"}, config=config)
```

---

# **6. Summary Table**

| **Feature**       | **Without Command**  | **With Command**                       |
| ----------------- | -------------------- | -------------------------------------- |
| Default behavior  | Always follows edges | You decide what happens next           |
| Interrupts        | Not possible         | Supported via `Command(interrupt=...)` |
| Dynamic jumps     | Not possible         | Use `Command(goto="node")`             |
| State updates     | Return dict only     | Use `Command(update={...})`            |
| Human-in-the-loop | Hard                 | Simple and clean                       |

---

# **7. Key Takeaways**

* **`Command` is the control system** for LangGraph.
* Use it when:

  * You need **human approval** → `interrupt`
  * You want to **jump dynamically** → `goto`
  * You need to **update state + change flow** → `update + goto`
* For **simple workflows**, return a dict directly.
* For **complex workflows** → use **`Command`**.

---

# **8. When to Use Command vs `interrupt()`**

| **Scenario**                   | **Use `interrupt()`** ✅ | **Use `Command`** 🧩 |
| ------------------------------ | ----------------------- | -------------------- |
| Simple pause/resume            | ✅ Yes                   | Optional             |
| Jumping to a specific node     | ❌ Not possible          | ✅ Use `goto`         |
| Combining pause + state update | ❌ Not possible          | ✅ Fully supported    |
| Complex compiled workflows     | ❌ Not suitable          | ✅ Best choice        |

---

## **Final Thoughts**

The **`Command` class** is powerful when:

* You’re building **human-in-the-loop** systems.
* You need **dynamic control** over graph execution.
* You want **interrupts + jumps + state updates** in one place.


---

In [1]:

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import TypedDict

class State(TypedDict):
    text: str

def node_a(state: State): 
    print("Node A")
    return Command(
        goto="node_b", 
        update={
            "text": state["text"] + "a"
        }
    )

def node_b(state: State): 
    print("Node B")
    return Command(
        goto="node_c", 
        update={
            "text": state["text"] + "b"
        }
    )


def node_c(state: State): 
    print("Node C")
    return Command(
        goto=END, 
        update={
            "text": state["text"] + "c"
        }
    )

graph = StateGraph(State)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)

graph.set_entry_point("node_a")


app = graph.compile()

response = app.invoke({
    "text": ""
})

Node A
Node B
Node C


In [2]:
response

{'text': 'abc'}